** takes a pickle created by 'hotline_to_pickle.py' **

In [1]:
## Import necessary modules
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import date2num, AutoDateFormatter, AutoDateLocator, WeekdayLocator, MonthLocator, DayLocator, DateLocator, DateFormatter
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
from matplotlib.ticker import AutoMinorLocator
import numpy as np
import datetime, calendar
from datetime import datetime
import matplotlib.patches as mpatches
%matplotlib tk

## Import data frome pickle generated from hotline_to_pickle.py
data = pd.read_pickle('/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/entwuerfe/plotlib_and_pickles/hotlinepickle_aktuell.pkl')

In [2]:
def rearrange(df):
    index_filter=df.loc[['tix']] ## select all hours-index rows
    
    hourse=list(range(0,25))  ## these is from 0 to 24 (25 entries!) and maps the hour indices
    mh=index_filter[hourse]  ## columns with half- and hour steps were named 0-24
    oldcols=mh.columns.values.tolist() ## list of the names of the hourse
    newcols=mh.loc['tix'].drop_duplicates().loc['tix'].values.tolist() ## one row of the hour indices provides the new column names
    cols=dict(zip(oldcols,newcols)) ## old column names get mapped to new columns names

    mhnew=df.rename(columns=cols).drop('tix') ## new dataframe has the old 'tix' entries as column names; 'tix' rows are dropped since not needed any more
    mhnew=mhnew.reset_index().set_index(['xlday']).rename(columns={'index':'status'}) ## new index will be the excel date numbers, old index is moved to a column named 'status'
    
    return mhnew, newcols

In [3]:
def timerange(*,df, years, months=range(1,13), weeks=range(1,53), weekdays=['Mon','Tue','Wed','Thu','Fri','Sat','Sun']):
    resultat=df.loc[(df['year'].isin(years)) & (df['month'].isin(months)) & (df['week'].isin(weeks)) & (df['weekday'].isin(weekdays))]
    frame_range=(resultat.date.min(),resultat.date.max())
    return resultat,frame_range

In [4]:
def summarize(frame):                ## at this place, we dont need the hourly figures, only day sums
    sumframe=frame.drop(hcols,axis=1)
    return sumframe

In [5]:
def hourize(frame):
    ndays=frame.index.nunique()
    anfr=frame[frame.status == 'angekommen'][hcols].sum().to_frame(name='angekommen')
    vbfr=frame[frame.status == 'verbunden'][hcols].sum().to_frame(name='verbunden')
    vlfr=frame[frame.status == 'verloren'][hcols].sum().to_frame(name='verloren')
    hs=pd.concat([anfr,vbfr,vlfr],axis=1)
    hs['sla']=(hs['verbunden']/hs['angekommen']).round(decimals=2)
    hs=hs.reset_index().rename(columns = {'index':'stunde'})
    return hs,ndays

In [6]:
def averagize(plotframe):
    pf = summarize(plotframe) ## just add everything and return total numbers
    pf['dayofweek']=pd.to_datetime(pf['date']).dt.strftime('%u').astype(int) ## add a weekday number

    gpf=pf.groupby(['weekday','status']).agg({'summa':'sum'}).unstack() # zu Mo,Di,Mi[..],So zusammenfassen und einen 2. Index für status anlegen
    gpf.columns=gpf.columns.droplevel()
    gpf['sla']=gpf['verbunden']/gpf['angekommen']

    pf2=pf.groupby(['weekday','status']).agg({'weekday':{'howmany':'count'},'date':{'datemin':'min','datemax':'max'},'dayofweek':{'dayofweek':'first'}})
    pf2.index=pf2.index.droplevel(1)
    pf2.columns=pf2.columns.droplevel(0)
    pf2=pf2.drop_duplicates()

    gpf=pd.concat([gpf,pf2], axis=1)
    gpf=gpf.sort_values(by='dayofweek')
    return gpf


In [7]:
def fplot(frame,style='default'):

    x_=frame.loc[frame['status'] == 'angekommen'].date.values
    x2_=frame.loc[frame['status'] == 'angekommen'].date
    y_an=frame.loc[frame['status'] == 'angekommen'].summa.values
    y_vb=frame.loc[frame['status'] == 'verbunden'].summa.values
    y_vl=frame.loc[frame['status'] == 'verloren'].summa.values
    y_sla=frame.loc[frame['status'] == 'servicelevel'].summa.values*100
    sla_c="#FF4D7D"
    an_c="#B5B5B5"
    vb_c="#008EC4"
    vl_c="#AC003A"
    slamin=y_sla.min()
    anmax=y_an.max()
    fstyles={'woche':(5,3),'monat':(12,6),'jahr':(20,5)}
    lsz={'woche':8,'monat':10,'jahr':10}
    fs=fstyles[style]
    
    def titgen(style):
        if style == 'woche':
            mi=pf.date.min().strftime('Anzahl Calls KW%W / %Y')
            ma=pf.date.max().strftime('Anzahl Calls KW%W / %Y')
            if mi == ma:
                tit=str(mi)
            else:
                tit=str(mi+' bis '+ma) 
        elif style == 'monat':
            mi=pf.date.min().strftime('Anzahl Calls %m %B %Y')
            ma=pf.date.max().strftime('Anzahl Calls %m %B %Y')
            if mi == ma:
                tit=str(mi)
            else:
                tit=str(mi+' bis '+ma)
        elif style == 'jahr':
            mi=pf.date.min().strftime('Anzahl Calls %Y')
            tit=str(mi)
        return tit
    tit=titgen(style)
            
    
    fig=plt.figure(figsize=(fs))
    fig.suptitle(tit)
    
    
    ########## SUBPLOT 0, Horiz. Line #############
    
    ax=plt.subplot(111)
    ax.plot(x_,y_sla,ls='None')
    if slamin < 55:
        ax.set_ylim(slamin-10,101)
    else:
        ax.set_ylim(50,101)
    ax.axhline(y=80,color=sla_c,ls='--',alpha=0.6, label='80%')
    

    ########## SUBPLOT 1, 3 Bars #############
   
    ax1=ax.twinx()
    ax1.yaxis.tick_left()
    if anmax < 100:
        ax1.set_ylim(0,100)
    anbar=ax1.bar(x_,y_an,width=0.8,color=an_c,label='angekommen')
    vnbar=ax1.bar(x_,y_vb,width=0.6,color=vb_c,label='verbunden')
    vlbar=ax1.bar(x_,y_vl,width=0.4,color=vl_c,label='lost')
    for bar in vnbar:
        xpos=bar.get_x()
        heig=bar.get_height()
        ax1.text(xpos+0.3,heig+4.3, s=str(int(heig)),ha='center',color=vb_c, weight='bold')
    for bar in vlbar:
        xpos=bar.get_x()
        heig=bar.get_height()
        if heig > 0:
            ax1.text(xpos+0.2,heig+1.7, s=str(int(heig)),ha='center',color=vl_c, weight='bold')
    
    ########## SUBPLOT 2, Scatter SLA above bars #############
    
    ax2=ax.twinx()
    pydt=[date2num(i) for i in x_]
    
    condition=['#4BE81B' if i>80 else '#E8344C' for i in y_sla]
    condsize=[10 if i>80 else 40 for i in y_sla]
    
    scat=ax2.scatter(pydt,y_sla, c=condition, marker='x', s=condsize, label='sla/tag')
    if slamin < 55:
        ax2.set_ylim(slamin-10,101)
    else:
        ax2.set_ylim(50,101)
    for ix,val in enumerate(y_sla):
        if val < 80:
            ax2.text(pydt[ix],ax2.get_ylim()[1]+1, s=str(round(val))+'%',ha='center',color=sla_c,size=lsz[style])

    
    #############    End of plotting. Now set labels, ticks etc.   ###############
    
    ## turn off the labels of the downmost plot. needs to be done after all plots are finished
        
    mjf={'monat':DateFormatter('%a,%d.%m.'),'woche':DateFormatter('%a,%d.%m.'),'jahr':DateFormatter('%m')}
    mnf={'monat':DateFormatter('%d.'),'woche':DateFormatter('%d.%m.'),'jahr':DateFormatter('%d')}
    
    xtick_mj_locator = WeekdayLocator(byweekday=MO)    
    xtick_mn_locator = WeekdayLocator(byweekday=[TU,WE,TH,FR,SA,SU])
    
    ax.xaxis.set_major_locator(xtick_mj_locator)
    ax.xaxis.set_major_formatter(mjf[style])
    
    ax.xaxis.set_minor_locator(xtick_mn_locator)
    ax.xaxis.set_minor_formatter(mnf[style])
    
    if style == 'monat':
        ax.tick_params(axis='x', which='major', labelsize=8, length=10, pad=5, colors='#003249')
        ax.tick_params(axis='x', which='minor', labelsize=8)
    elif style == 'woche':
        ax.tick_params(axis='x', which='both', labelsize=8, length=2, pad=2)
        
    ax.tick_params(axis='y',which='both',left='off',right='off',labelleft='off',labelright='off') # First: disable the copied ax.labels
   

    
    # disable spines for the downmost plot so the spines won't overlay
    ax.spines['right'].set_visible(False) 
    ax.spines['left'].set_visible(False) 
    ax.spines['top'].set_visible(False) 
    ax.spines['bottom'].set_visible(False) 
    
    ax.margins(0.01)
    
    #### use the middle plot for the y-axis ticks and label on the left side and gridlines
    ax1.set_ylabel('Anzahl Calls', fontsize=10)
    ax1.yaxis.set_label_position('left')
    ax1.spines['right'].set_visible(False) 
    ax1.spines['left'].set_visible(False) 
    ax1.spines['top'].set_visible(False) 
    ax1.spines['bottom'].set_visible(False) 

    
    y_int=ax1.yaxis.get_majorticklocs()[1]  #|
    if y_int > 10:                          #|
        min_locs=AutoMinorLocator(y_int/5)  #| this will return steps of 4 for 20
    elif y_int <= 10:                       #| and steps of 2 for 10
        min_locs=AutoMinorLocator(5)        #|
            
    ax1.yaxis.set_minor_locator(min_locs)
    ax1.set_axisbelow(True)
    ax1.yaxis.grid(b=True, which='major', color=vb_c, linestyle='-')
    ax1.yaxis.grid(b=True, which='minor', color='#E8E6BF', linestyle='--', alpha=1, lw=0.5)
    ax1.tick_params(axis='y',which='both',left='on',right='off',labelleft='on',labelright='off', labelsize=lsz[style])
    ax1.tick_params(axis='x',which='both',bottom='off',top='off',labelbottom='off',labeltop='off') #xticks off -> ax.xticks
    
    #### use the top plot for the y-axis ticks and label on the right side
    ax2.tick_params(axis='y',which='both',left='off',right='on',labelleft='off',labelright='on', labelsize=lsz[style])
    ax2.set_ylabel('Prozent SLA', fontsize=10)
    ax2.yaxis.set_label_position('right')
    ax2.spines['right'].set_color(sla_c)
    ax2.spines['left'].set_color(vb_c)
    ax2.spines['left'].set_lw(2)
    ax2.tick_params(axis='x',which='both',bottom='off',top='off',labelbottom='off',labeltop='off') #xticks off -> ax.xticks
    
    handles = [anbar,vnbar,vlbar,scat]
    labels  = ['# angekommen','# verbunden','# verloren','sla']
    leg=ax2.legend(handles,labels,bbox_to_anchor=(0.8, 1), loc='upper left',prop={'size':7}, framealpha=0.8)  
    
    #plt.close(fig)
    
    return fig,tit.replace(' ','').replace('/','_')

In [8]:
def awplot(frame):

    x_=frame.dayofweek.values
    y_an=frame.angekommen.values
    y_vb=frame.verbunden.values
    y_vl=frame.verloren.values
    
    y_an_av=(frame.angekommen.values/frame.howmany.values).round(decimals=1)
    y_vb_av=(frame.verbunden.values/frame.howmany.values).round(decimals=1)
    y_vl_av=(frame.verloren.values/frame.howmany.values).round(decimals=1)
    
    y_sla=frame.sla.values*100
    sla_c="#FF4D7D"
    an_c="#B5B5B5"
    vb_c="#008EC4"
    vl_c="#AC003A"
    #slamin=y_sla.min()
    slamin=frame.sla.min()*100
    anmax=y_an.max()

    mi=frame.datemin.min().strftime('%d.%m.%Y')
    ma=frame.datemax.max().strftime('%d.%m.%Y')
    tit=str('Calls nach Wochentagen von '+mi+' bis '+ma)
    filetit=str('durschnittl_wochentage'+mi+'-'+ma).replace('.','_')
        
    fig=plt.figure(figsize=(7,5))
    fig.suptitle(tit)
    
    
    ########## SUBPLOT 0, Horiz. Line #############
    
    ax=plt.subplot(111)
    ax.plot(x_,y_sla,ls='None')
    
    if slamin < 55:
        ax.set_ylim(slamin-10,101)
        print('set sla axis to lower than '+str(slamin))
    else:
        ax.set_ylim(50,101)

    ax.axhline(y=80,color=sla_c,ls='--',alpha=0.6, label='80%')
    

    ########## SUBPLOT 1, 3 Bars #############
   
    ax1=ax.twinx()
    ax1.yaxis.tick_left()
    if anmax < 100:
        ax1.set_ylim(0,100)
    else:
        ax1.set_ylim(0,anmax+(anmax/10))

    anbar=ax1.bar(x_,y_an,width=0.8,color=an_c,label='angekommen')
    vbbar=ax1.bar(x_,y_vb,width=0.6,color=vb_c,label='verbunden')
    vlbar=ax1.bar(x_,y_vl,width=0.4,color=vl_c,label='lost')
    
    count=0
    for bar in anbar:
        xpos=bar.get_x()
        heig=bar.get_height()
        ax1.text(xpos+0.4,heig+25, s='Ø '+str(y_an_av[count]),ha='center',color=an_c,bbox=dict(facecolor='white',pad=1.5, edgecolor=an_c))
        count=count+1
    for bar in vbbar:
        xpos=bar.get_x()
        heig=bar.get_height()
        ax1.text(xpos+0.3,heig+4.3, s=str(int(heig)),ha='center',color=vb_c, weight='bold')
    for bar in vlbar:
        xpos=bar.get_x()
        heig=bar.get_height()
        if heig > 0:
            ax1.text(xpos+0.2,heig+1.7, s=str(int(heig)),ha='center',color=vl_c, weight='bold')
    
    ########## SUBPLOT 2, Scatter SLA above bars #############
    
    ax2=ax.twinx()
    #pydt=[date2num(i) for i in x_]
    
    condition=['#4BE81B' if i>80 else '#E8344C' for i in y_sla]
    condsize=[10 if i>80 else 40 for i in y_sla]
    
    scat=ax2.scatter(x_,y_sla, c=condition, marker='x', s=condsize, label='sla/tag')
    if slamin < 55:
        ax2.set_ylim(slamin-10,101)
    else:
        ax2.set_ylim(50,101)
    for ix,val in enumerate(y_sla):
        if val < 80:
            ax2.text(x_[ix],ax2.get_ylim()[1]+1, s=str(round(val))+'%',ha='center',color=sla_c,size=8)

    
    #############    End of plotting. Now set labels, ticks etc.   ###############
    
    ax.set_xticks(frame.dayofweek.values)
    ax.set_xticklabels(frame.index)
    ax.tick_params(axis='x', which='both', labelsize=8, length=2, pad=2)
    ax.tick_params(axis='y',which='both',left='off',right='off',labelleft='off',labelright='off') # First: disable the copied ax.labels
   

    
    # disable spines for the downmost plot so the spines won't overlay
    ax.spines['right'].set_visible(False) 
    ax.spines['left'].set_visible(False) 
    ax.spines['top'].set_visible(False) 
    ax.spines['bottom'].set_visible(False) 
    
    ax.margins(0.01)
    
    #### use the middle plot for the y-axis ticks and label on the left side and gridlines
    ax1.set_ylabel('Anzahl Calls', fontsize=10)
    ax1.yaxis.set_label_position('left')
    ax1.spines['right'].set_visible(False) 
    ax1.spines['left'].set_visible(False) 
    ax1.spines['top'].set_visible(False) 
    ax1.spines['bottom'].set_visible(False) 

    
    y_int=ax1.yaxis.get_majorticklocs()[1]  #|
    if y_int >= 50:                          #|
        min_locs=AutoMinorLocator(y_int/10)
    elif y_int > 10:                          #|
        min_locs=AutoMinorLocator(y_int/5)  #| this will return steps of 4 for 20
    elif y_int <= 10:                       #| and steps of 2 for 10
        min_locs=AutoMinorLocator(5)        #|

    ax1.yaxis.set_minor_locator(min_locs)
    ax1.set_axisbelow(True)
    ax1.yaxis.grid(b=True, which='major', color=vb_c, linestyle='-')
    ax1.yaxis.grid(b=True, which='minor', color='#E8E6BF', linestyle='--', alpha=1, lw=0.5)
    ax1.tick_params(axis='y',which='both',left='on',right='off',labelleft='on',labelright='off', labelsize=8)
    ax1.tick_params(axis='x',which='both',bottom='off',top='off',labelbottom='off',labeltop='off') #xticks off -> ax.xticks
    
    #### use the top plot for the y-axis ticks and label on the right side
    ax2.tick_params(axis='y',which='both',left='off',right='on',labelleft='off',labelright='on', labelsize=8)
    ax2.set_ylabel('Prozent SLA', fontsize=10)
    ax2.yaxis.set_label_position('right')
    ax2.spines['right'].set_color(sla_c)
    ax2.spines['left'].set_color(vb_c)
    ax2.spines['left'].set_lw(2)
    ax2.tick_params(axis='x',which='both',bottom='off',top='off',labelbottom='off',labeltop='off') #xticks off -> ax.xticks
    
    #ax1.legend(prop={'size':8})    
    
    #plt.close(fig)
    atxest = mpatches.Rectangle((0, 0), 1, 1, fc="white",edgecolor=an_c,label='av. angekommen')
    handles = [atxest,anbar,vbbar,vlbar,scat]
    labels  = ['Ø angekommen','# angekommen','# verbunden','# verloren','sla']
    leg=ax2.legend(handles,labels,bbox_to_anchor=(0.8, 1), loc='upper left',prop={'size':8}, framealpha=0.8)

    return fig,filetit

In [9]:
def hourplot(frame,vonbis,ndays):

    von,bis=vonbis
    x_=frame.index
    y_an_abs=frame.angekommen.values
    y_vb_abs=frame.verbunden.values
    y_vl_abs=frame.verloren.values   
    y_an=(y_an_abs/ndays).round(decimals=1)
    y_vb=(y_vb_abs/ndays).round(decimals=1)
    y_vl=(y_vl_abs/ndays).round(decimals=1)
 
    y_sla=frame.sla.values*100
    sla_c="#FF4D7D"
    an_c="#B5B5B5"
    vb_c="#008EC4"
    vl_c="#AC003A"
    slamin=frame.sla.min()*100
    anmax=y_an.max()
    lim=anmax+5
    tit=str('durchschnittl. Stunde von '+von.strftime('%d.%m.%y')+' bis '+bis.strftime('%d.%m.%y'))

    
    fig=plt.figure(figsize=(12,5))
    fig.suptitle(tit)
    
    
    ########## SUBPLOT 0, Horiz. Line #############
    
    ax=plt.subplot(111)
    ax.plot(x_,y_sla,ls='None')
    if slamin < 55:
        ax.set_ylim(slamin-10,101)
        print('set sla axis to lower than '+str(slamin))
    else:
        ax.set_ylim(50,101)
    ax.axhline(y=80,color=sla_c,ls='--',alpha=0.6, label='80%')
    
    ########## SUBPLOT 1, 3 Bars #############
   
    ax1=ax.twinx()
    ax1.yaxis.tick_left()

    ax1.set_ylim(0,lim)
    mheigh=lim-((lim-anmax)/2)
    anbar=ax1.bar(x_,y_an,width=0.8,color=an_c,label='# angekommen')
    vbbar=ax1.bar(x_,y_vb,width=0.6,color=vb_c,label='# verbunden')
    vlbar=ax1.bar(x_,y_vl,width=0.4,color=vl_c,label='# lost')
    count=0
    for bar in anbar:
        xpos=bar.get_x()
        heig=bar.get_height()
        if heig > 0:
            if count%2 == 0:
                atxt=ax1.text(xpos+0.4,anmax+(lim/15), s='Ø'+str(heig),ha='center',color=an_c,fontsize=(lim*0.6),bbox=dict(facecolor='white',pad=1, edgecolor=an_c))
            else:
                atxt=ax1.text(xpos+0.4,anmax+(lim/15)-1, s='Ø'+str(heig),ha='center',color=an_c,fontsize=(lim*0.6),bbox=dict(facecolor='white',pad=1, edgecolor=an_c))
        count=count+1
    count=0
    for bar in vbbar:
        xpos=bar.get_x()
        heig=bar.get_height()
        if heig > 0:
            ax1.text(xpos+0.3,heig+(lim/20)-0.5, s=str(int(y_vb_abs[count])),ha='center',color=vb_c, weight='bold')
        count=count+1
    count=0
    for bar in vlbar:
        xpos=bar.get_x()
        heig=bar.get_height()
        if heig > 0:
            ax1.text(xpos+0.2,heig+(lim/20)-0.5, s=str(int(y_vl_abs[count])),ha='center',color=vl_c, weight='bold')
        count=count+1
    
    ########## SUBPLOT 2, Scatter SLA above bars #############
    
    ax2=ax.twinx()
    
    condition=['#4BE81B' if i>80 else '#E8344C' for i in y_sla]
    condsize=[10 if i>80 else 40 for i in y_sla]
    
    scat=ax2.scatter(x_,y_sla, c=condition, marker='x', s=condsize, label='sla/tag')
    if slamin < 55:
        ax2.set_ylim(slamin-10,101)
    else:
        ax2.set_ylim(50,101)
    for ix,val in enumerate(y_sla):
        if val < 80:
            ax2.text(x_[ix],ax2.get_ylim()[1]+1, s=str(round(val))+'%',ha='center',color=sla_c,size=8)
    
    #### Here go the ticks
    ax.set_xticks(frame.index[::2,]) # every second element from index, starting at 0
    ax.set_xticks(frame.index[1::2,], minor=True) # every second element from index, starting at 1, set to minor ticks
    ax.set_xticklabels(frame.stunde[::2,])
    ax.set_xticklabels(frame.stunde[1::2,], minor=True)
    #ax.set_xticks(frame.index)
    #ax.set_xticklabels(frame.stunde)
    ax.tick_params(axis='x', which='major', labelsize=7, length=2, pad=2)
    ax.tick_params(axis='x', which='minor', labelsize=7, length=20, pad=2)
    ax.tick_params(axis='y',which='both',left='off',right='off',labelleft='off',labelright='off') # First: disable the copied ax.labels
   

    # disable spines for the downmost plot so the spines won't overlay
    ax.spines['right'].set_visible(False) 
    ax.spines['left'].set_visible(False) 
    ax.spines['top'].set_visible(False) 
    ax.spines['bottom'].set_visible(False) 
    
    ax.xaxis.grid(b=True, which='major', color=an_c, linestyle='-')
    ax.margins(0.01)
    
    #### use the middle plot for the y-axis ticks and label on the left side and gridlines
    ax1.set_ylabel('Anzahl Calls pro Stunde', fontsize=10)
    ax1.yaxis.set_label_position('left')
    ax1.spines['right'].set_visible(False) 
    ax1.spines['left'].set_visible(False) 
    ax1.spines['top'].set_visible(False) 
    ax1.spines['bottom'].set_visible(False) 

    
    y_int=ax1.yaxis.get_majorticklocs()[1]  #|
    if y_int >= 50:                         #|
        min_locs=AutoMinorLocator(y_int/10) #|
    elif y_int > 10:                        #|
        min_locs=AutoMinorLocator(y_int/5)  #| this will return steps of 4 for 20
    elif y_int <= 10:                       #| and steps of 2 for 10
        min_locs=AutoMinorLocator(5)        #|

    ax1.yaxis.set_minor_locator(min_locs)
    ax1.set_axisbelow(True)
    ax1.yaxis.grid(b=True, which='major', color=vb_c, linestyle='-')
    ax1.yaxis.grid(b=True, which='minor', color='#E8E6BF', linestyle='--', alpha=1, lw=0.5)
    ax1.tick_params(axis='y',which='both',left='on',right='off',labelleft='on',labelright='off', labelsize=8)
    ax1.tick_params(axis='x',which='both',bottom='off',top='off',labelbottom='off',labeltop='off') #xticks off -> ax.xticks
    
    #### use the top plot for the y-axis ticks and label on the right side
    ax2.tick_params(axis='y',which='both',left='off',right='on',labelleft='off',labelright='on', labelsize=8)
    ax2.set_ylabel('Prozent SLA', fontsize=10)
    ax2.yaxis.set_label_position('right')
    ax2.spines['right'].set_color(sla_c)
    ax2.spines['left'].set_color(vb_c)
    ax2.spines['left'].set_lw(2)
    ax2.tick_params(axis='x',which='both',bottom='off',top='off',labelbottom='off',labeltop='off') #xticks off -> ax.xticks
    
    atxest = mpatches.Rectangle((0, 0), 1, 1, fc="white",edgecolor=an_c,label='av. angekommen')
    handles = [atxest,anbar,vbbar,vlbar,scat]
    labels  = ['Ø angekommen','# angekommen','# verbunden','# verloren','sla']
    leg=ax2.legend(handles,labels,bbox_to_anchor=(0.1, 1), loc='upper right',prop={'size':8}, framealpha=0.8)
    
    #plt.close(fig)
    
    return fig,tit.replace(' ','_').replace('/','_').replace('.','_')

# end of function definitions

In [10]:
ndata,hcols=rearrange(data) ## rearranged dataframe with hour_indices as colums and list of hour_index-columns for future use
maxcallsperday=ndata.loc[ndata.status == 'angekommen'].summa.max() ## hoechste Anzahl Calls im pickle
ndata.head()

,status,month,year,00:00-00:30,00:30-01:30,01:30-02:30,02:30-03:30,03:30-04:30,04:30-05:30,05:30-06:30,...,19:30-20:30,20:30-21:30,21:30-22:30,22:30-23:30,23:30-00:00,summa,day,date,week,weekday
xlday,,,,,,,,,,,,,,,,,,,,,
42688.0,angekommen,11,2016,0,0,0,0,0,0,1,...,0,0,1,0,0,321,14,2016-11-14,46,Mon
42688.0,verbunden,11,2016,0,0,0,0,0,0,0,...,0,0,0,0,0,221,14,2016-11-14,46,Mon
42688.0,verloren,11,2016,0,0,0,0,0,0,1,...,0,0,1,0,0,100,14,2016-11-14,46,Mon
42688.0,servicelevel,11,2016,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,0,NaN,NaN,0.688474,14,2016-11-14,46,Mon
42689.0,angekommen,11,2016,0,0,0,0,0,0,1,...,1,0,0,0,0,152,15,2016-11-15,46,Tue


### time range filter:
Arguments must be passed as a list or as a range. Example:

**timerange(df=ndata,years=[2016,2017],months=[3],weeks=[6],weekdays=['Sun','Sun'])**

Note the double entry for a single weekday!
Leaving parameters empty gives default values (=all). Months and Weeks may be mutually exclusive and raise exceptions

#### styles:
fplot function will take a style= argument
styles may be 'woche', 'monat' or 'year', increasing scale and adjusting text and label sizes

In [14]:
## absolute Calls nach Datum
## Woche oder Monat angeben

for i in range (28,29):
    plotframe,plotrange = timerange(df=ndata,years=[2017],weeks=[i]) ## filter out a given time range in 'weeks' or 'months'
    if not plotframe.empty:
        pf = summarize(plotframe) ## just add everything and return total numbers
        plotted,titel=fplot(pf,style='woche') # style='woche' oder 'monat' oder 'jahr'
#        path='/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/ce_teamleitung/plots/absolut/Kalenderwochen/'
#        plotted.savefig(path+titel,ext='png')


In [ ]:
## durchschnittliche Woche für einen angegebenen Zeitraum
# plotframe,plotrange = timerange(df=ndata,years=[2017],months=range(7,8))  ## filter out a given time; range(x,y) or list[x,y,z]
# if not plotframe.empty:
#    av_week=averagize(plotframe)
#    wtage,titel=awplot(av_week)
#    path='/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/ce_teamleitung/plots/nach_Wochentag/'
#    wtage.savefig(path+titel,ext='png')


In [ ]:
## durchschittlicher Tag nach Stunden

#plotframe,plotrange = timerange(df=ndata,years=[2017],weeks=range(29,30))
#if not plotframe.empty:
#    vonbis=plotframe.date.min(),plotframe.date.max()
#    hrs,ndays=hourize(plotframe)
#    hp,titel=hourplot(hrs,vonbis,ndays)
#    path='/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/ce_teamleitung/plots/nach_Stunde/'
#    hp.savefig(path+titel,ext='png')